In [1]:
!pip install faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 23.9 MB/s eta 0:00:00


In [2]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

In [3]:
import pandas as pd

In [4]:
data =[
    ['How are you?','general'],
    ['How is the fall of stocks due to new AI Model','finance'],
    ['How to make Masala Dosa?','cooking'],
    ['How do i create a new email id?','tech support'],
    ['What are the new movies releasing in Pongal','entertainment'],
    ['How to avoid diabeties?','health'],
    ['Cheapest flights from India to Germany','travel'],
    ['Recommend a good sweet shop nearby','food'],
    ['When and where will FIFA World Cup be conducted','ports'],
    ['It was quite a hectic day today','personal emotion']
]

df = pd.DataFrame(data,columns=['text','category'])

In [5]:
df.head()

,text,category
0,How are you?,general
1,How is the fall of stocks due to new AI Model,finance
2,How to make Masala Dosa?,cooking
3,How do i create a new email id?,tech support
4,What are the new movies releasing in Pongal,entertainment


In [6]:
from sentence_transformers import SentenceTransformer
text = df['text']
encoder = SentenceTransformer("paraphrase-mpnet-base-v2") #pretrained model and optimises generating a semantically meaningful sentence embeddings
embeddings = encoder.encode(text)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
embeddings.shape

(10, 768)

In [11]:
import faiss
vector_dimensions = embeddings.shape[1]
index = faiss.IndexFlatL2(vector_dimensions) # works only for L2 and we normalize our vectors to one units
faiss.normalize_L2(embeddings)
index.add(embeddings)

In [23]:
import numpy as np
search_text = 'How to avoid Arithritis'
search_vector = encoder.encode(search_text)
new_vector = np.array([search_vector])
faiss.normalize_L2(new_vector)

In [24]:
dist, ann = index.search(new_vector, k = 4) #aproximate nearest neighbour
results=pd.DataFrame({'distances' : dist[0],'ann': ann[0]})
df_merged = pd.merge(results,df,left_on = 'ann',right_index = True)
df_merged.head()

,distances,ann,text,category
0,1.011956,5,How to avoid diabeties?,health
1,1.648469,2,How to make Masala Dosa?,cooking
2,1.672224,7,Recommend a good sweet shop nearby,food
3,1.708303,6,Cheapest flights from India to Germany,travel


In [ ]:
# My search_text is closest to helth category.